In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_worldfootball(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    matches = []

    for table in soup.find_all("table"):
        for row in table.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) < 6:
                continue

            date = cols[0].get_text(strip=True)
            home = cols[2].get_text(strip=True)
            away = cols[4].get_text(strip=True)
            score = cols[5].get_text(strip=True)

            if ":" not in score:
                continue

            hg, ag = score.split(":")

            matches.append({
                "date": date,
                "home_team": home,
                "away_team": away,
                "home_goals": int(hg),
                "away_goals": int(ag)
            })

    return pd.DataFrame(matches)


df = scrape_worldfootball("https://www.worldfootball.net/schedule/eng-premier-league-2023-2024/")
print("Rows:", len(df))
print(df.head())




Rows: 0
Empty DataFrame
Columns: []
Index: []
